Fullname: Lê Hữu Sang

Student ID: 22120310

# HW3: CUDA Memory

**To compile your file, you can use this command:** \
`nvcc filename.cu -o execute_filename` \
***You can use Vietnamese to anwser the questions***

You can add/delete cells as you see fit. Just don't delete the cells with the Teacher's words.

---

Below is the command to install `Nsight system` on Colab. You just need to run the cell.

In [1]:
# Here is the command to install Nsight system on Colab. You just need to run the cell.
!apt-get update
!apt-get install -y cuda-nsight-systems-12-2

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,827 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

- The above installation will be remove when colab restart its runtime. It should be fine for this homework because we don't use Nsight system that much.
- But if you want to persistent install Nsight system, you mount space from your Google Drive to your Colab VM. Read [Here](https://stackoverflow.com/questions/76784746/how-to-use-nsys-in-google-colab) for guidelines how to do thhis.

In [3]:
from numba import cuda

major, minor = cuda.get_current_device().compute_capability
print(f"GPU compute capability: {major}.{minor}")

GPU compute capability: 7.5


In [7]:
from PIL import Image
import matplotlib.pyplot as plt

def show_pnm_side_by_side(file_list, titles=None, figsize=(15, 5)):
    n = len(file_list)
    plt.figure(figsize=figsize)

    for i, path in enumerate(file_list):
        try:
            img = Image.open(path)
        except Exception as e:
            print(f"Cannot open {path}: {e}")
            continue

        plt.subplot(1, n, i + 1)
        cmap = "gray" if img.mode == "L" else None
        plt.imshow(img, cmap=cmap)
        title = titles[i] if titles and i < len(titles) else path
        plt.title(title)
        plt.axis("off")

    plt.tight_layout()
    plt.show()


---

## Exercises 1

In [5]:
!nvcc -arch=sm_{major}{minor} HW3_P1.cu -o HW3_P1

In [6]:
!./HW3_P1 in.pnm out.pnm

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15828320256 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x16, grid size 32x32
Kernel time: 0.696928 ms
Error: 0.000703

Kernel 2, block size 16x16, grid size 32x32
Kernel time: 0.347808 ms
Error: 0.000703

Kernel 3, block size 16x16, grid size 32x32
Kernel time: 0.269824 ms
Error: 0.000703


## Nhận xét chi tiết

### 1) Kernel 1 – đọc dữ liệu thô từ Global Memory
Phiên bản này mỗi thread phải tự lấy toàn bộ 81 giá trị từ global memory để thực hiện convolution.  
Dù GPU có cache hỗ trợ, lượng truy cập vẫn rất lớn, và **bandwidth DRAM trở thành giới hạn chính**.  
**Ưu điểm:** dễ cài đặt.

---

### 2) Kernel 2 – dùng Shared Memory để chứa vùng tile
Ở phiên bản này, cả block hợp tác nạp toàn bộ ô dữ liệu đầu vào (bao gồm cả biên) lên shared memory **một lần duy nhất**.  
Phần dữ liệu này sau đó được **tái sử dụng cho tất cả 256 pixels output** trong cùng block.

**Ưu điểm:**

- Số lần đọc từ DRAM giảm xuống chỉ còn một phần nhỏ so với kernel 1  
- Truy cập shared memory nhanh hơn nhiều vì độ trễ thấp và nằm ngay trong SM  
- Chi phí đồng bộ (`__syncthreads()`) chỉ một vài lần và nhỏ hơn rất nhiều so với lượng dữ liệu tiết kiệm được  

**Kết quả thực tế:** thời gian giảm gần **2×**.

---

### 3) Kernel 3 – combination của Shared Memory + Constant Memory
Phiên bản cuối vừa tận dụng tile trong shared memory, vừa đưa toàn bộ **filter** vào constant memory.

**Ưu điểm:**

- Bộ lọc kích thước nhỏ, phù hợp với constant memory  
- Mỗi bước tính, toàn bộ thread trong warp đọc cùng một hệ số  
- GPU có cơ chế **broadcast** cho cả warp chỉ trong một giao dịch  
- Hiệu quả cache gần như tối đa (hit rate rất cao)

**Kết quả:** phần đọc hệ số filter từ global memory gần như bị loại bỏ hoàn toàn, thời gian tiếp tục giảm thêm khoảng **20–25%** so với kernel 2.


## Exercises 2

In [19]:
!nvcc -arch=sm_{major}{minor} HW3_P2.cu -o HW3_P2

In [20]:
!./HW3_P2

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 8
  Elements per vector: 4194304 (16.00 MB)
  Total data: 128.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 26.30 ms

=== BREADTH-FIRST PIPELINE ===
Time: 15.70 ms

=== DEPTH-FIRST PIPELINE ===
Time: 15.68 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     26.30 ms
Breadth-First:  15.70 ms  (1.68x speedup)
Depth-First:    15.68 ms  (1.68x speedup)

Depth-First vs Breadth-First: 1.00x


In [21]:
!./HW3_P2 16

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 16
  Elements per vector: 4194304 (16.00 MB)
  Total data: 256.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 51.06 ms

=== BREADTH-FIRST PIPELINE ===
Time: 33.23 ms

=== DEPTH-FIRST PIPELINE ===
Time: 31.07 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     51.06 ms
Breadth-First:  33.23 ms  (1.54x speedup)
Depth-First:    31.07 ms  (1.64x speedup)

Depth-First vs Breadth-First: 1.07x


In [22]:
!./HW3_P2 32

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 32
  Elements per vector: 4194304 (16.00 MB)
  Total data: 512.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 100.11 ms

=== BREADTH-FIRST PIPELINE ===
Time: 68.56 ms

=== DEPTH-FIRST PIPELINE ===
Time: 61.76 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     100.11 ms
Breadth-First:  68.56 ms  (1.46x speedup)
Depth-First:    61.76 ms  (1.62x speedup)

Depth-First vs Breadth-First: 1.11x


In [23]:
# Generate report with nsight system
!nsys profile ./HW3_P2

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 8
  Elements per vector: 4194304 (16.00 MB)
  Total data: 128.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 25.94 ms

=== BREADTH-FIRST PIPELINE ===
Time: 15.49 ms

=== DEPTH-FIRST PIPELINE ===
Time: 15.50 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     25.94 ms
Breadth-First:  15.49 ms  (1.67x speedup)
Depth-First:    15.50 ms  (1.67x speedup)

Depth-First vs Breadth-First: 1.00x
Generating '/tmp/nsys-report-a084.qdstrm'
[1/1] [========================100%] report1.nsys-rep
Generated:
    /content/report1.nsys-rep
